In [1]:
from datetime import date
import pandas as pd
import yaml
from sqlalchemy import create_engine
import numpy as np

In [2]:
# Cargar configuración
with open('../config.yml', 'r') as f:
    config = yaml.safe_load(f)
    config_fuente = config['fuente']
    config_bodega = config['bodega']

# Crear conexiones
url_fuente = f"postgresql://{config_fuente['user']}:{config_fuente['password']}@{config_fuente['host']}:{config_fuente['port']}/{config_fuente['dbname']}"
url_bodega = f"postgresql://{config_bodega['user']}:{config_bodega['password']}@{config_bodega['host']}:{config_bodega['port']}/{config_bodega['dbname']}"

fuente_conn = create_engine(url_fuente)
bodega_conn = create_engine(url_bodega)

In [4]:
# Consulta SQL para obtener datos de cliente junto con su tipo y ciudad
query = """
SELECT 
    c.cliente_id,
    c.nombre,
    c.nit_cliente,
    tc.nombre as tipo_cliente,
    c.sector,
    c.email,
    c.telefono,
    c.direccion,
    c.nombre_contacto,
    ci.nombre as ciudad
FROM cliente c
LEFT JOIN tipo_cliente tc ON c.tipo_cliente_id = tc.tipo_cliente_id
LEFT JOIN ciudad ci ON c.ciudad_id = ci.ciudad_id
"""

In [5]:
# Leer datos de la fuente
dim_cliente = pd.read_sql(query, fuente_conn)

In [6]:
dim_cliente.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   cliente_id       27 non-null     int64 
 1   nombre           27 non-null     object
 2   nit_cliente      27 non-null     object
 3   tipo_cliente     27 non-null     object
 4   sector           27 non-null     object
 5   email            27 non-null     object
 6   telefono         27 non-null     object
 7   direccion        27 non-null     object
 8   nombre_contacto  27 non-null     object
 9   ciudad           27 non-null     object
dtypes: int64(1), object(9)
memory usage: 2.2+ KB


In [7]:
print(dim_cliente.head())

   cliente_id                           nombre nit_cliente      tipo_cliente  \
0           1                        Cliente 2          25  Persona Juridica   
1           2                        Cliente 1         123  Persona Juridica   
2           6       CLINICA  DEPORTIVA DEL SUR     24390-3  Persona Juridica   
3          19  HOSPITAL ORTOPEDICO DE COLOMBIA     8301821  Persona Juridica   
4           8      CLINICA NEFROLOGOS  DE CALI   5017350-8  Persona Juridica   

       sector     email   telefono           direccion    nombre_contacto  \
0           S  algo.com  327-00000  Calle 100 No 25-18  Cristiano Ronaldo   
1  industrial  algo.com  327-00000  Calle 100 No 25-18  Cristiano Ronaldo   
2       salud  algo.com  327-00000  Calle 100 No 25-18  Cristiano Ronaldo   
3       salud  algo.com  327-00000  Calle 100 No 25-18  Cristiano Ronaldo   
4       salud  algo.com  327-00000  Calle 100 No 25-18  Cristiano Ronaldo   

  ciudad  
0   CALI  
1   CALI  
2   CALI  
3   CALI  
4

In [8]:
#Agregar fecha de carga
dim_cliente['saved'] = date.today()

In [11]:
# Cargar en la bodega de datos
dim_cliente.to_sql('dim_cliente', bodega_conn, if_exists='replace', index_label='key_dim_cliente')

27